In [1]:
import json
import csv
import copy

In [2]:
lookaround = 4

# characters to use to denote special parts of words
outside = " "
boundary = "#"
self = "_"
anything = "*"
padding = outside * (lookaround-1)

# letters to target to fix
replaceable = "äöü"
unreplace = {
    "ae": "ä",
    "oe": "ö",
    "ue": "ü"
}

def pad(x):
    return padding+boundary+x+boundary+padding

def is_valid(s):
    return all(ord(c) < 128 or c in replaceable for c in s)

counts = {}
words = []

with open("deu_mixed-typical_2011_1M-words.txt") as tsvfile: # http://wortschatz.uni-leipzig.de/en/download
    reader = csv.reader(tsvfile, delimiter='\t', quotechar=None)
    for row in reader:
        word = row[2].strip().lower()
        counts[pad(word)] = int(row[3])
        words.append(word)

words = [x.lower() for x in words if not x.isupper()] # remove acronyms (hopefully)
words = [x for x in words if x.isalpha() and is_valid(x)] # remove non-letter stuff

targets = [pad(x) for x in words]
len(targets)

363935

In [3]:
targets[:50:10]

['   #der#   ', '   #ein#   ', '   #sich#   ', '   #zu#   ', '   #bei#   ']

In [4]:
probs_before = {}
probs_after = {}

def insert_sub(dic, string, count):
    if len(string) == 0:
        return
    elif len(string) == 1:
        match = string[0]
        # group together all non-replaceable end characters
        if match not in replaceable:
            match = anything
        # initialize if not existant
        if match not in dic:
            dic[match] = {}
        # use a special leaf node and increment the count of that frequency
        dic[match][self] = dic[match].get(self, 0) + count
    elif string[0] == outside:
        insert_sub(dic, string[1:], count)
    elif len(string) > 1:
        match = string[0]
        if match not in dic:
            dic[match] = {self: 0}
        insert_sub(dic[match], string[1:], count)

for word in targets:
    for pos, letter in enumerate(word):
        insert_sub(probs_before, word[pos-lookaround : pos+1], counts[word])
        insert_sub(probs_after, word[pos : pos+lookaround+1][::-1], counts[word])

print("done")

done


In [5]:
def simplify(dic):
    removable = []
    for key in dic:
        if type(dic[key]) is int:
            if dic[key] == 0:
                removable.append(key)
            elif len(dic) == 1:
                return dic[key]
        else:
            val = simplify(dic[key])
            if val is not None:
                dic[key] = val
    for key in removable:
        del dic[key]

simplify(probs_before)
simplify(probs_after)

In [6]:
package = {
    "version": 1,
    "params": {
        "lookaround": lookaround,
        "outside": outside,
        "boundary": boundary,
        "self": self,
        "anything": anything,
        "padding": padding,
        "replaceable": replaceable,
        "unreplace": unreplace
    },
    "before": probs_before,
    "after": probs_after
}

with open("ext/src/dump.json", 'w') as outfile:
    json.dump(package, outfile, separators=(',', ':'))

len(json.dumps(package, separators=(',', ':')))

3038895

In [7]:
def explore(pointer, string, orig, target):
    for letter in string:
        if letter == outside:
            continue
        if letter not in pointer:
            return 0
        pointer = pointer[letter]
    max_count = 1
    target_count = 1
    orig_count = 1
    if type(pointer) is not int:
        for possibility in pointer:
            if possibility == self:
                continue
            freq = extract_freq(pointer[possibility])
            if freq > max_count:
                max_count = freq
        if target in pointer:
            target_count = extract_freq(pointer[target])
        for i in range(len(orig)):
            letter = orig[i]
            if i == len(orig) - 1:
                letter = anything
            if letter not in pointer:
                break
            pointer = pointer[letter]
            orig_count = max(extract_freq(pointer), 1)
    return target_count/max_count

def extract_freq(dic):
    freq = dic
    if type(freq) is not int:
        if self in dic:
            freq = dic[self]
        else:
            freq = 0
    return freq

def avg(a, b):
    return (a+b)/2

In [8]:
def suggest(word):
#     for key in unreplace:
#         word = word.replace(key, unreplace[key])
#     return (word, 1)
    word = pad(word)
    string = word.lower()
    confidence = 1
    # check if there is a potential target
    if any(x in string for x in unreplace):
        for pos in range(2, len(string)-3):
            orig = string[pos : pos+2]
            if orig in unreplace:
                target = unreplace[orig]
                prob_b = explore(probs_before, string[pos-lookaround-1 : pos], orig, target)
                prob_a = explore(probs_after, string[pos+2 : pos+lookaround+2][::-1], orig, target)
                prob = max(prob_a, prob_b)
                if prob > 0.02:
                    word = word[:pos]+target+word[pos+2:]
                    string = string[:pos]+target+string[pos+2:]
                    pos -= 2
                    confidence *= prob
                else:
                    confidence *= (1 - prob)
    return (word[lookaround:len(word)-lookaround], confidence)

## Benchmarking below

total_uml = 0
correct_uml = 0
total_non_uml = 0
correct_non_uml = 0

for word in words:
    umlaut = any(x in word for x in replaceable)
    given = word
    count = counts[pad(word)]
    for key in unreplace:
        given = given.replace(unreplace[key], key)
    result = suggest(given)[0]
    if umlaut:
        total_uml += count
        if result == word:
            correct_uml += count
    else:
        total_non_uml += count
        if result == word:
            correct_non_uml += count

print("Umlauts changed correctly:       ", round(correct_uml/total_uml, 5))
print("Non-umlauts changed incorrectly: ", round((1-(correct_non_uml/total_non_uml)), 5))

precision = correct_uml / total_uml
recall = (correct_uml+correct_non_uml) / (total_uml+total_non_uml)
f1 = 2 * (precision * recall) / (precision + recall)
print("F-1 score:                       ", round(f1, 5))

example = "muesste Muehe zuerst neue Mueller frueher anhaengen drangehaengt Nuernberg \
abfuellen Fussball Bruehe ruehrend Zuerich loeten zoegern anloeten aehnlich frueher \
Kuehe Erlaeuterungen erneuern genuegend Zuege Nussnacker".split(" ")
output = "\n"

for word in example:
    suggestion = suggest(word)
    output += "{} ({:.4f}) ".format(suggestion[0], suggestion[1])

print(output)

Umlauts changed correctly:        0.9058
Non-umlauts changed incorrectly:  0.0026
F-1 score:                        0.9431

müsste (0.0474) Mühe (0.0474) zuerst (0.9859) neue (0.9999) Müller (0.0474) früher (0.1473) anhängen (0.4901) drangehängt (0.3098) Nürnberg (0.8122) abfüllen (0.0455) Fussball (1.0000) Brühe (0.0817) rührend (0.2050) Zuerich (0.9859) löten (0.0361) zögern (0.0347) anloeten (0.9985) ähnlich (1.0000) früher (0.1473) Kühe (0.0376) Erläuterungen (0.3591) erneuern (0.9998) genügend (0.0281) Züge (0.0398) Nussnacker (1.0000) 
